## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-19-10-46-50 +0000,bbc,'Shameful' and 'terrible look' - the chaos tha...,https://www.bbc.com/sport/football/articles/c7...
1,2026-01-19-10-40-46 +0000,bbc,Ecstasy and now a day off - Senegal fans aroun...,https://www.bbc.com/news/articles/ce3ekz4yx21o...
2,2026-01-19-10-40-23 +0000,bbc,Why does Trump want Greenland and what could i...,https://www.bbc.com/news/articles/c74x4m71pmjo...
3,2026-01-19-10-38-58 +0000,wapo,Trump plans to charge $1 billion for permanent...,https://www.washingtonpost.com/national-securi...
4,2026-01-19-10-37-45 +0000,bbc,Government pulls Hillsborough Law debate after...,https://www.bbc.com/news/articles/c5yvldj47v9o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,54
18,greenland,22
37,tariff,15
38,threat,12
116,ice,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
169,2026-01-18-21-05-32 +0000,bbc,EU weighs response to Trump's tariff threat ov...,https://www.bbc.com/news/articles/c87r41j4n4jo...,111
208,2026-01-18-16-09-10 +0000,cbc,Canada 'concerned' about Trump's Greenland tar...,https://www.cbc.ca/news/politics/canada-greenl...,108
101,2026-01-19-02-38-00 +0000,wsj,Precious Metals Climb as Trump Plans Tariff on...,https://www.wsj.com/finance/precious-metals-cl...,108
170,2026-01-18-20-42-50 +0000,bbc,Starmer holds phone call with Trump over Green...,https://www.bbc.com/news/articles/cwyn90l1dneo...,107
50,2026-01-19-08-45-00 +0000,wsj,Eurozone Bond Yields Decline Amid Trump’s Tari...,https://www.wsj.com/finance/jgb-futures-fall-a...,106


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
169,111,2026-01-18-21-05-32 +0000,bbc,EU weighs response to Trump's tariff threat ov...,https://www.bbc.com/news/articles/c87r41j4n4jo...
139,58,2026-01-19-00-00-06 +0000,nypost,At least 20 killed in high-speed train derailm...,https://nypost.com/2026/01/18/world-news/at-le...
198,49,2026-01-18-17-50-47 +0000,nypost,Minneapolis Mayor Jacob Frey rips ICE’s immigr...,https://nypost.com/2026/01/18/us-news/minneapo...
174,33,2026-01-18-20-13-23 +0000,nypost,Hundreds of thousands of french fries wash up ...,https://nypost.com/2026/01/18/world-news/hundr...
110,32,2026-01-19-01-42-43 +0000,nypost,Brazen NYC carjacker swipes vehicle where blin...,https://nypost.com/2026/01/18/us-news/brazen-n...
94,27,2026-01-19-03-40-25 +0000,nypost,Florida man randomly shot three tourists dead ...,https://nypost.com/2026/01/18/us-news/florida-...
65,27,2026-01-19-06-40-23 +0000,bbc,China hits growth goal after exports defy US t...,https://www.bbc.com/news/articles/cgk8zd287myo...
178,27,2026-01-18-20-06-05 +0000,cbc,Trump's 'Board of Peace' proposal met with ske...,https://www.cbc.ca/news/world/gaza-ceasefire-p...
145,27,2026-01-18-23-24-23 +0000,nypost,Pregnant mother of two ID’d among six women ki...,https://nypost.com/2026/01/18/world-news/pregn...
58,26,2026-01-19-07-35-27 +0000,nypost,Trump says building a ‘New York Stock Exchange...,https://nypost.com/2026/01/19/us-news/trump-sa...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
